In [ ]:
!pip install pdfminer.six

In [ ]:
import os
import json
from pdfminer.high_level import extract_text

# 폴더 내의 모든 PDF 파일에서 안전 지침 추출하고 PDF별로 배열에 저장
def extract_safety_guidelines_from_folder(folder_path, output_file):
    results = {}

    # 폴더 내의 모든 파일 순회
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)

            # 각 PDF 파일에서 텍스트 추출
            try:
                text = extract_text(pdf_path)
                # 전체 텍스트를 하나의 문자열로 저장
                # 줄바꿈 및 여러 공백 제거하여 깔끔하게 정리
                cleaned_text = ' '.join(text.split())

                # PDF 파일명을 키로, 텍스트를 값으로 저장
                pdf_name = os.path.splitext(filename)[0]  # 확장자 제거
                results[pdf_name] = cleaned_text

                print(f"'{filename}' 파일의 텍스트를 추출했습니다. (길이: {len(cleaned_text)} 자)")
            except Exception as e:
                print(f"'{filename}' 파일 처리 중 오류 발생: {e}")

    # 결과를 JSON 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"총 {len(results)}개 PDF의 텍스트를 '{output_file}'에 저장했습니다.")
    return results

# 사용 예시
folder_path = "/content/drive/MyDrive/pdfs"  # 실제 PDF 폴더 경로로 변경하세요
output_file = "/content/drive/MyDrive/pdf_texts.json"  # 결과를 저장할 파일명
pdf_texts = extract_safety_guidelines_from_folder(folder_path, output_file)

# 수행완료 시 결과 예시
# {
#   "F.C.M 교량공사 안전보건작업 지침": "F.C.M(Free Cantilever Method) 교량공사는 주로 계곡이나 깊은 하천을 횡단하는 교량 시공법으로, 양쪽 교각에서 시작하여 중앙부에서 만나도록 시공하는 공법입니다. 이 공법은 높은 고소작업이 수반되므로 다음과 같은 안전보건 지침을 준수해야 합니다. 작업자는 반드시 안전모, 안전대, 안전화 등 개인보호구를 착용해야 합니다. 고소작업 시 추락방지용 안전망을 설치하고 작업발판은 견고하게 고정해야 합니다. 강풍이나 폭우 시에는 작업을 중지하고 안전한 장소로 대피해야 합니다...",

#   "I.L.M 교량공사 안전보건작업 지침": "I.L.M(Incremental Launching Method) 교량공사는 육상에서 상부구조물을 제작한 후 교각 위로 밀어내는 방식으로 시공하는 공법입니다. 이 공법은 주로 도심지나 환경보전지역에서 사용되며 다음과 같은 안전보건 지침을 준수해야 합니다. 교량 상부구조를 제작하는 제작장은 평탄하게 조성하고 작업공간을 충분히 확보해야 합니다. 발사(Launching) 작업 시 구조물의 안정성을 수시로 확인하고 작업자가 구조물과 고정 구조물 사이에 끼이지 않도록 주의해야 합니다. 유압장치 및 기계장비는 사용 전 반드시 점검하고 안전장치가 정상 작동하는지 확인해야 합니다..."
# }

'I.L.M 교량공사 안전보건작업 지침.pdf' 파일의 텍스트를 추출했습니다. (길이: 12061 자)
'F.C.M 교량공사 안전보건작업 지침.pdf' 파일의 텍스트를 추출했습니다. (길이: 12859 자)
총 2개 PDF의 텍스트를 '/content/drive/MyDrive/pdf_texts.json'에 저장했습니다.


In [ ]:
# 간단한 3개 카테고리로 시작 -> 학습용
categories = ['위험경고', '예방조치', '점검사항']

# 수동 라벨링 예시 (각 카테고리당 30-50개)
labeled_examples = [
    {'text': '교각 브라켓 해체작업 시 근로자가 추락할 위험이 있습니다.', 'category': '위험경고'},
    {'text': '거푸집은 시공 중 변형이 발생되지 않도록 강성과 정밀도를 확보하여야 한다.', 'category': '예방조치'},
    {'text': '이동식 작업대차는 정기 및 수시 점검을 통하여 이상유무를 확인하여야 한다.', 'category': '점검사항'}
]

# Flair 형식으로 변환
with open('train.txt', 'w', encoding='utf-8') as f:
    for example in labeled_examples:
        f.write(f"__label__{example['category']} {example['text']}\n")

# 결과 예시(train.txt) -> __label__ , 카테고리 , 우리 넣은  모델 훈련 용 데이터셋

# __label__위험경고 교각 브라켓 해체작업 시 근로자가 추락할 위험이 있습니다.
# __label__예방조치 거푸집은 시공 중 변형이 발생되지 않도록 강성과 정밀도를 확보하여야 한다.
# __label__점검사항 이동식 작업대차는 정기 및 수시 점검을 통하여 이상유무를 확인하여야 한다.
# __label__위험경고 크레인 작업 반경 내에 근로자의 출입을 금지시켜야 한다.
# __label__예방조치 추락위험이 있는 장소에서 콘크리트 타설 작업 시에는 안전모, 안전대 등의 보호구를 항상 착용하여야 한다.
# __label__점검사항 압출잭의 압출용량이 충분한지를 확인하고, 압출력 및 압출상태 등 압출장비에 대한 이상유무도 점검하여야 한다.

In [ ]:
!pip install scikit-learn

In [ ]:
# 이게 실제로 우리가 사용할 학습데이터들.

import pandas as pd
from sklearn.model_selection import train_test_split

# CSV 파일에서 데이터 로드
df = pd.read_csv("/content/drive/MyDrive/processed_construction-safety-guidelines.csv")

# NaN 값 처리 및 텍스트 정제
df = df.dropna(subset=['지침내용', '지침유형', '공사종류'])  # 필수 열에 NaN이 있는 행 제거
df['지침내용'] = df['지침내용'].str.replace('\n', ' ').str.strip()  # 줄바꿈 제거 및 공백 정리

# 추가 CSV 파일들 리스트
additional_files = [ "/content/drive/MyDrive/labels_강경민.csv" , "/content/drive/MyDrive/건설 데이터라벨링_최민혜2.CSV" , "/content/drive/MyDrive/part4_sentence.csv" ,"/content/drive/MyDrive/aid_5_데이터_박비원.csv"]

# 나머지 파일들도 같은 방식으로 처리해서 하나의 리스트에 담기
dfs = [df]  # 첫 번째 df도 포함
for file in additional_files:
    temp_df = pd.read_csv(file)
    temp_df = temp_df.dropna(subset=['지침내용', '지침유형', '공사종류'])
    temp_df['지침내용'] = temp_df['지침내용'].str.replace('\n', ' ').str.strip()
    dfs.append(temp_df)

# 모든 데이터프레임 합치기
combined_df = pd.concat(dfs, ignore_index=True)
print("combined_df의 행 수 , ")
print(len(combined_df))

# 방법 1: loc 또는 iloc을 사용하여 원본 데이터프레임 직접 수정
for idx, row in combined_df.iterrows():
    if row["지침유형"] == "예방 조치":
        combined_df.loc[idx, "지침유형"] = "예방조치"
    elif row["지침유형"] == "위험 경고":
        combined_df.loc[idx, "지침유형"] = "위험경고"
    elif row["지침유형"] == "점검 사항":
        combined_df.loc[idx, "지침유형"] = "점검사항"

# 방법 2: replace 메서드 사용 (더 간단하고 효율적)
combined_df["지침유형"] = combined_df["지침유형"].replace({
    "예방 조치": "예방조치",
    "위험 경고": "위험경고",
    "점검 사항": "점검사항"
})

    # 1. TypeError: tuple indices must be integers, not str when accessing cursor.fetchall results -> row[1]["지침유형"] 로 써야 함.
    # 2. iterrows()쓰면 변화값이 원본에 반영이 안 됨 -> loc , iloc[idx, "column명"] = 으로 바꾸던지 replce 사용 필수.

indices_to_drop = combined_df[combined_df["지침유형"] == "지침유형"].index
combined_df = combined_df.drop(indices_to_drop) # 또는 .isin()

print(combined_df["지침유형"].unique())

count = 0;

for idx, row in combined_df.iterrows():
    if row["지침유형"] == "위험경고":
      count += 1;

print(count)

# 먼저 train과 temp로 나누기 (80% 학습, 20% 임시)
train_df, temp_df = train_test_split(combined_df, test_size=0.2, random_state=42)

print(train_df['지침유형'].value_counts())

# temp를 다시 dev와 test로 나누기 (각각 10%)
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# 각 분류 차원에 대한 별도 파일 생성
with open('type_train.txt', 'w', encoding='utf-8') as f:
    for _, row in train_df.iterrows():
        text = row['지침내용']
        type_category = row['지침유형']
        f.write(f"__label__{type_category} {text}\n")

with open('type_dev.txt', 'w', encoding='utf-8') as f:
    for _, row in dev_df.iterrows():
        text = row['지침내용']
        type_category = row['지침유형']
        f.write(f"__label__{type_category} {text}\n")

with open('type_test.txt', 'w', encoding='utf-8') as f:
    for _, row in test_df.iterrows():
        text = row['지침내용']
        type_category = row['지침유형']
        f.write(f"__label__{type_category} {text}\n")

with open('construction_train.txt', 'w', encoding='utf-8') as f:
    for _, row in train_df.iterrows():
        text = row['지침내용']
        construction_category = row['공사종류']
        f.write(f"__label__{construction_category} {text}\n")

with open('construction_dev.txt', 'w', encoding='utf-8') as f:
    for _, row in dev_df.iterrows():
        text = row['지침내용']
        type_category = row['공사종류']
        f.write(f"__label__{type_category} {text}\n")

with open('construction_test.txt', 'w', encoding='utf-8') as f:
    for _, row in test_df.iterrows():
        text = row['지침내용']
        type_category = row['공사종류']
        f.write(f"__label__{type_category} {text}\n")

print(f"처리된 데이터 수: {len(combined_df)}")

combined_df의 행 수 , 
1049
['예방조치' '위험경고' '점검사항']
117
지침유형
예방조치    419
점검사항    325
위험경고     94
Name: count, dtype: int64
처리된 데이터 수: 1048


In [ ]:
!head -n 5 /content/sample_data/type_train_dev_test/type_train.txt

head: cannot open '/content/sample_data/type_train_dev_test/type_train.txt' for reading: No such file or directory


<1주차 정리>

- 머신러닝 , 딥러닝 , 지도학습 , 비지도학습 , 강화학습 기본 설명

- 안전지침 pdf들이 모여있는 폴더를 순회하면서 ,
   키 : pdf의 제목 , value: 추출한 문장들 형식의 추출결과 저장

- train.csv 를 읽어서 필요한 열의 정보들을 추출

- 안전지침만을 자동으로 추출하는 엔진 제작 과정을 설명

- 안전지침 분류 모델로 사용할 Flair 모듈에 주입할 훈련용 라벨링 데이터 제작 설명

해야할 것 : 개인별 랩탑 세팅 , 개인별 라벨링 한 데이터 수집.

PyTorch(torch)는 딥러닝 프레임워크로, Flair 라이브러리가 내부적으로 PyTorch를 기반으로 작동하기 때문에 필요합니다.

Flair의 핵심 기능들은 모두 PyTorch로 구현되어 있습니다:

모델 구조: TextClassifier 같은 Flair의 모델들은 PyTorch의 nn.Module을 상속받아 구현됩니다.

학습 과정: 모델 학습은 PyTorch의 옵티마이저와 손실 함수를 사용합니다.

GPU 가속: PyTorch를 통해 CUDA를 활용한 GPU 학습이 가능합니다.

임베딩 처리: 트랜스포머 모델을 포함한 다양한 임베딩도 PyTorch를 통해 처리됩니다.

따라서 Flair를 사용하려면 반드시 PyTorch가 설치되어 있어야 합니다. PyTorch가 없으면 Flair 라이브러리 자체가 작동하지 않습니다.

In [ ]:
# 1. 필요한 패키지 설치
!pip install flair transformers torch sentencepiece kobert-transformers -q
# 넘파이 - transformer 버전 조합 때문에 고민을 했었는데 tourch 를 설치 함으로써 딥러닝이 요하는 연산을 다 커버할 수 있게 되었다.
!pip install -U huggingface_hub

# 2. 디렉토리 생성
!mkdir -p /content/sample_data/type_train_dev_test
!mkdir -p /content/sample_data/construction_train_dev_test
!mkdir -p models/type-classifier
!mkdir -p models/construction-classifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 3. 이미 생성된 파일들을 적절한 디렉토리로 이동
!mv type_train.txt /content/sample_data/type_train_dev_test/
!mv type_dev.txt /content/sample_data/type_train_dev_test/
!mv type_test.txt /content/sample_data/type_train_dev_test/
!mv construction_train.txt /content/sample_data/construction_train_dev_test/
!mv construction_dev.txt /content/sample_data/construction_train_dev_test/
!mv construction_test.txt /content/sample_data/construction_train_dev_test/

# 4. 필요한 라이브러리 및 모듈 임포트
import torch
from flair.data import Corpus # 텍스트 데이터 처리를 위한 Corpus 클래스
from flair.datasets import ClassificationCorpus # 분류 작업을 위한 특화된 코퍼스 클래스
from flair.embeddings import TransformerDocumentEmbeddings # 트랜스포머 기반 임베딩 (BERT 등)
from flair.models import TextClassifier # 텍스트 분류 모델
from flair.trainers import ModelTrainer # 모델 학습을 위한 트레이너

# 5. GPU 확인 # colab 설정에서 하드웨어 가속기로 cpu가 아닌 gpu를 사용하도록 설정 변경.
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")

CUDA 사용 가능: True
GPU 이름: Tesla T4


type_corpus = ClassificationCorpus("/content/sample_data/type_train_dev_test",  # 데이터 디렉토리 경로
                                 train_file='type_train.txt',  # 학습 데이터 파일
                                 dev_file='type_dev.txt',  # 검증 데이터 파일
                                 test_file='type_test.txt')  # 테스트 데이터 파일:

이 코드의 역할 ->

1.파일 경로 지정

- 첫 번째 인자 ("/content/sample_data/type_train_dev_test")는 데이터 파일들이 위치한 기본 디렉토리 경로입니다 ,

- train_file, dev_file, test_file 매개변수는 각각 학습, 검증, 테스트에 사용될 데이터 파일의 이름을 지정합니다.

2.데이터 로드 및 전처리:

- 지정된 파일들에서 텍스트와 라벨을 읽어옵니다.(아까 txt 파일을 열어서 봤듯이 __label__라벨명-실제내용 이런 형태로 쭉 쓰여져 있다.)
- 텍스트와 라벨을 Flair의 내부 형식으로 변환합니다.
- 학습/검증/테스트 세트를 각각 분리하여 저장합니다.

3.라벨 처리:

- 파일에서 'label'로 시작하는 라벨을 추출하고, 기본적으로 'class' 타입의 라벨로 변환합니다.
- 이렇게 생성된 코퍼스에서 나중에 make_label_dictionary()를 호출하면 라벨 사전을 얻을 수 있습니다.


결론 : type_corpus 객체는 모델 학습, 평가, 예측 등의 과정에서 데이터를 제공하는 역할 -> 모델이 이해할 수 있도록 우리가 만든 텍스트와 라벨쌍을 그 내부 구조에 맞춰 바꿔주는 역할.



---------------------------------------------------------

type_classifier = TextClassifier(document_embeddings,
                                label_dictionary=type_corpus.make_label_dictionary(label_type= 'class'), label_type='class')
 이 코드의 역할 :

1.분류기 생성:

- TextClassifier 인스턴스를 생성합니다.

2.임베딩 모델 연결:

- document_embeddings 매개변수를 통해 텍스트를 벡터로на 변환하는 임베딩 모델을 분류기에 연결합니다.

3.라벨 사전 설정:

- type_corpus.make_label_dictionary(label_type='class')는 코퍼스에서 'class' 타입으로 지정된 모든 라벨을 수집하여 사전(dictionary) 형태로 만듭니다. 이 사전은 분류기가 어떤 클래스들을 예측할 수 있는지 정의합니다.

4.라벨 타입 지정:

- label_type='class' 매개변수는 분류기가 'class' 타입의 라벨을 예측할 것임을 지정합니다. 이는 코퍼스에서 사용한 라벨 타입과 일치해야 합니다.

- (결과적으로 분류된 그 카테고리의 타입이 class일 것을 의미함 -> 당연하다. 왜냐면 class 타입의 라벨들을 corpus에서 받아서 dictionary의 인덱스로 설정해 놓은 셈이기에.)

결론:

- 이 분류기는 입력 텍스트를 document_embeddings를 사용해 벡터로 변환한 후, 그 벡터를 바탕으로 라벨 사전에 정의된 클래스 중 하나로 분류하는 작업을 수행합니다. 'class' 타입으로 지정된 라벨들(원래 파일에서는 'label'로 시작하는 태그들)을 예측하게 됩니다.

# 랜덤 서치 + 교차 검증 방식:

하이퍼파라미터 공간 정의:

- 학습률: [0.001, 0.1] 연속 공간
- 배치 크기: [8, 16, 32, 64]
- 에폭 수: [5, 10, 20]


랜덤 조합 생성:

- 정의된 공간에서 15 ~ 20개 하이퍼 파라미터 조합을 무작위로 추출


각 랜덤 조합에 교차 검증 적용:

- 각 조합마다 5-fold 교차 검증 수행
- 각 조합별로 5개 성능 점수의 평균 계산


최적 조합 선택:

- 교차 검증 평균 성능이 가장 높은 하이퍼 파라미터 조합 선택


최종 모델링:

- 선택된 최적 조합으로 전체 훈련 데이터 사용해 최종 모델 학습


## 교차검증이란?

- 교차 검증(Cross-validation)은 모델 성능을 더 안정적으로 평가하고 과적합을 방지하는 기법입니다.
교차 검증 기본 개념:

데이터 분할:
- 전체 데이터셋을 여러 개의 부분(폴드)으로 나눕니다.
반복 학습 및 평가: 각 반복에서 하나의 폴드를 검증용으로 사용하고 나머지를 훈련용으로 사용합니다.
성능 평균화: 모든 폴드에 대한 평가 결과를 평균내어 최종 성능을 측정합니다.

가장 일반적인 형태 - k-fold 교차 검증:

예를 들어 5-fold 교차 검증에서는:

데이터를 5개 부분으로 나눕니다.
5번의 실험을 수행합니다:

1번째 실험: 1번 폴드를 검증용, 2~5번 폴드를 훈련용으로 사용

2번째 실험: 2번 폴드를 검증용, 1,3~5번 폴드를 훈련용으로 사용

...이런 식으로 5번째 실험까지 진행


5번의 실험에서 얻은 성능 지표의 평균값을 최종 성능으로 사용합니다.

교차 검증의 장점:

- 더 안정적인 성능 평가: 데이터 분할에 따른 우연의 영향을 줄여줍니다.
- 전체 데이터 활용: 모든 데이터 포인트가 훈련과 검증에 모두 사용됩니다.
- 과적합 감지: 폴드 간 성능 차이가 크면 과적합의 징후일 수 있습니다.
- 하이퍼파라미터 튜닝 신뢰성 향상: 단일 검증 세트보다 더 안정적인 평가를 제공합니다.

- 텍스트 분류 모델 튜닝에서는 5-fold나 10-fold 교차 검증을 많이 사용하며, 각 하이퍼파라미터 조합에 대해 교차 검증 점수의 평균을 비교하여 최적의 조합을 선택합니다.

### 교차검증 수행 예시(구체적)

5-fold 교차 검증의 정확한 과정:

- 데이터 분할: 전체 데이터를 5개의 동일한 크기의 폴드로 나눕니다.

  각 폴드는 전체 데이터의 20%를 포함


- 1차 반복:

  검증용: 1번 폴드(전체의 20%)
  훈련용: 2,3,4,5번 폴드(전체의 80%)
  모델을 처음부터 학습 후 1번 폴드로 검증


- 2차 반복:

  검증용: 2번 폴드(전체의 20%)
  훈련용: 1,3,4,5번 폴드(전체의 80%)
  새 모델을 처음부터 학습 후 2번 폴드로 검증


- 3~5차 반복:

  각 폴드를 한 번씩 검증용으로 사용
  항상 나머지 4개 폴드(80%)로 훈련


- 최종 결과:

  5번의 검증 성능 평균을 해당 하이퍼파라미터의 성능으로 결정



각 폴드가 검증용으로 사용될 때, 그 폴드 전체가 검증에 사용됩니다(폴드 내에서 다시 나누지 않음). 이렇게 모든 데이터가 한 번씩 검증에 사용되고, 모든 데이터 포인트가 모델 평가에 기여하게 됩니다.

#올바른 머신러닝 평가 파이프라인:

1. 최초 데이터 분리:

  전체 데이터를 훈련/검증용(80%)과 테스트용(20%)으로 분리
  테스트 데이터는 "봉인"하여 최종 평가 전까지 절대 사용하지 않음



2. 하이퍼파라미터 최적화 (랜덤 서치 + 교차 검증):

  훈련/검증용 데이터(80%)만 가지고 교차 검증 수행
  각 하이퍼파라미터 조합마다 k-fold 교차 검증으로 안정적인 평가
  평균 성능이 가장 높은 하이퍼파라미터 조합 선택



3. 최종 모델 학습:

  선택된 최적 하이퍼파라미터로
  전체 훈련/검증용 데이터(80%)를 사용해 최종 모델 학습


4.  최종 평가:

  처음에 분리해 놓은 테스트 데이터(20%)로
  최종 모델의 성능을 한 번만 평가
  이 성능이 가장 신뢰할 수 있는 일반화 성능 지표


In [ ]:
import random
import numpy as np
from sklearn.model_selection import StratifiedKFold
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.data import Sentence
from flair.datasets import ClassificationCorpus
import os
import tempfile
import gc
import torch

def random_search_with_cross_validation(corpus, document_embeddings, n_trials=15, n_folds=2, label_type='class'): # 15 ~ 20 회가 랜덤 수행의 결과가 좋을 가능성이 높지만 , gpu시간을 고려해서 우선 15회로 설정.
    """
    랜덤 하이퍼파라미터 서치 + 교차 검증
    """
    # 데이터 준비
    sentences = corpus.train
    texts = [s.to_plain_string() for s in sentences]
    labels = [s.get_label(label_type).value for s in sentences]

    # 계층적 폴드 분할 -> labels-texts 를 균등하게(shuffle = true) fold 만큼의 카테고리로 나누기.
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    fold_indices = list(skf.split(texts, labels))

    # 하이퍼파라미터 범위 정의
    learning_rate_range = [0.005, 0.1]  # 학습률 범위
    batch_sizes = [8, 16, 32]  # 배치 크기 옵션
    epochs_range = [5, 12]  # 에폭 범위(이 범위를 넘어서는 수가 설정되면 바로 과적합 나옴.)

    # 결과 저장용
    best_score = 0
    best_params = None
    results = []

    # 랜덤 서치 시작
    for trial in range(n_trials):
        # 랜덤 하이퍼파라미터 생성
        lr = np.exp(random.uniform(np.log(learning_rate_range[0]),
                                  np.log(learning_rate_range[1])))
        batch_size = random.choice(batch_sizes)
        epochs = random.randint(epochs_range[0], epochs_range[1])

        params = {
            'learning_rate': lr,
            'batch_size': batch_size,
            'epochs': epochs
        }

        print(f"시도 {trial+1}/{n_trials}: lr={lr:.4f}, batch_size={batch_size}, epochs={epochs}")

        # 교차 검증 수행
        fold_scores = []
        valid_fold_count = 0  # 유효한 폴드 수 추적

        for fold_idx, (train_idx, val_idx) in enumerate(fold_indices): # fold_idx 는 해당 폴드가 몇번째인지 , (train_idx , val_idx) 는 해당 폴드에 지정된 np.array
            print(f"  폴드 {fold_idx+1}/{n_folds} 학습 중...")

            # 이 폴드의 훈련/검증 문장 추출
            train_sentences = [sentences[i] for i in train_idx]
            val_sentences = [sentences[i] for i in val_idx]

            # 임시 파일 및 코퍼스 생성 -> 그냥 label-texts 쌍만 우리가 만들어 놓은 상태라서 이를 학습용 파일 즉 flair이 사용할 수 있는 형태의 텍스트 임시 파일로 변형.
            with tempfile.TemporaryDirectory() as temp_dir:
                # 훈련/검증 파일 생성 -> 임시 디렉터리 생성 후 그 내에 임시 파일 생성 -> 각 순번의 훈련이 끝나면 삭제.
                train_file = os.path.join(temp_dir, "train.txt")
                dev_file = os.path.join(temp_dir, "dev.txt")

                # 훈련 파일 작성
                with open(train_file, 'w', encoding='utf-8') as f:
                    for s in train_sentences:
                        f.write(f"__label__{s.get_label(label_type).value} {s.to_plain_string()}\n")

                # 검증 파일 작성
                with open(dev_file, 'w', encoding='utf-8') as f:
                    for s in val_sentences:
                        f.write(f"__label__{s.get_label(label_type).value} {s.to_plain_string()}\n")

                # 임시 코퍼스 생성
                temp_corpus = ClassificationCorpus(temp_dir, train_file="train.txt",
                                                 dev_file="dev.txt", test_file=None)

                # 분류기 생성
                classifier = TextClassifier(document_embeddings,
                                          label_dictionary=temp_corpus.make_label_dictionary(label_type),
                                          label_type=label_type)

                # 트레이너 생성 및 학습
                trainer = ModelTrainer(classifier, temp_corpus)
                try:
                    result = trainer.train(
                        base_path=os.path.join(temp_dir, f"model"),
                        learning_rate=lr,
                        mini_batch_size=batch_size,
                        max_epochs=epochs
                    )
                    # 결과 출력해보기
                    print(f"  훈련 결과: {result}")

                    # 결과 구조에 따라 적절히 접근
                    # 현재 폴드에서 학습된 모델을 로드해서 직접 평가하는 것이 더 안전
                    current_fold_model_path = os.path.join(temp_dir, f"model/best-model.pt")

                    # 현재 폴드에서 학습된 모델로 직접 점수 계산
                    current_fold_model = TextClassifier.load(current_fold_model_path)

                    # 현재 폴드의 검증 세트에서 직접 평가
                    correct_predictions = 0
                    total_samples = 0

                    for sentence in val_sentences:
                        # 문장 복제 (레이블 정보 보존)
                        test_sentence = Sentence(sentence.to_plain_string())
                        true_label = sentence.get_label(label_type).value

                        # 예측
                        current_fold_model.predict(test_sentence)
                        pred_label = test_sentence.get_label('class').value

                        if pred_label == true_label:
                            correct_predictions += 1
                        total_samples += 1

                    # 현재 폴드의 정확도 계산
                    current_fold_accuracy = correct_predictions / total_samples if total_samples > 0 else 0
                    fold_scores.append(current_fold_accuracy)
                    valid_fold_count += 1  # 유효한 폴드 증가
                    print(f"  현재 폴드 정확도: {current_fold_accuracy:.4f}")

                    # 메모리 정리
                    del current_fold_model
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    gc.collect()

                except Exception as e:
                    print(f"  오류 발생: {e}")
                    # 오류 발생 시 점수를 추가하지 않음 (유효 폴드 수가 증가하지 않음)

        # 이 조합의 평균 점수 계산
        if valid_fold_count > 0:  # 유효한 폴드가 있을 때만 처리
            avg_score = sum(fold_scores) / valid_fold_count
            print(f"  평균 성능: {avg_score:.4f} (유효한 폴드: {valid_fold_count}/{n_folds})")

            results.append({
                'params': params,
                'score': avg_score,
                'fold_scores': fold_scores,
                'valid_folds': valid_fold_count
            })

            # 최고 성능 갱신
            if avg_score > best_score:
                best_score = avg_score
                best_params = params
        else:
            print(f"  이 하이퍼파라미터 조합에 대한 유효한 결과 없음")

    # 결과 정렬 및 출력
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)

    print("\n=== 랜덤 서치 결과 요약 ===")
    for i, result in enumerate(sorted_results[:5]):  # 상위 5개만 표시
        p = result['params']
        print(f"{i+1}위: lr={p['learning_rate']:.4f}, batch_size={p['batch_size']}, epochs={p['epochs']} "
              f"- 점수: {result['score']:.4f} (유효 폴드: {result['valid_folds']}/{n_folds})")

    print(f"\n최적 하이퍼파라미터: {best_params}, 점수: {best_score:.4f}")

    return best_params, sorted_results



construction_corpus = ClassificationCorpus("/content/sample_data/construction_train_dev_test",  # 데이터 디렉토리 경로
                                         train_file='construction_train.txt',  # 학습 데이터 파일
                                         dev_file='construction_dev.txt',  # 검증 데이터 파일
                                         test_file='construction_test.txt', # 테스트 데이터 파일
                                         )

document_embeddings = TransformerDocumentEmbeddings('klue/bert-base')


# 앞서 생성한 document_embeddings를 재사용 (모델 공유)
construction_classifier = TextClassifier(document_embeddings,
                                       label_dictionary=construction_corpus.make_label_dictionary(label_type= 'class'), label_type='class')  # 공사 유형 레이블 사전 생성



#랜덤서치 하이퍼파라미터 최적화 수행
best_params, results = random_search_with_cross_validation(
    corpus=construction_corpus,
    document_embeddings=document_embeddings,
    n_trials=2,  # 10번의 랜덤 시도 -> 즉!! "n-trials * (폴드수) " -> 만큼의 accuracy 측정이 있겠죠.
    n_folds=5     # 5-fold 교차 검증
)

2025-05-01 08:12:43,071 Reading data from /content/sample_data/construction_train_dev_test
2025-05-01 08:12:43,072 Train: /content/sample_data/construction_train_dev_test/construction_train.txt
2025-05-01 08:12:43,073 Dev: /content/sample_data/construction_train_dev_test/construction_dev.txt
2025-05-01 08:12:43,074 Test: /content/sample_data/construction_train_dev_test/construction_test.txt
2025-05-01 08:12:43,116 Initialized corpus /content/sample_data/construction_train_dev_test (label type name is 'class')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

...

2025-05-01 08:36:40,531 DEV : loss 0.03724370896816254 - f1-score (micro avg)  0.988
2025-05-01 08:36:40,573  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.002149116225804514]


2025-05-01 08:36:43,748 ----------------------------------------------------------------------------------------------------
2025-05-01 08:36:43,750 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  4.34it/s]

2025-05-01 08:36:46,360 
Results:
- F-score (micro) 0.9851
- F-score (macro) 0.9862
- Accuracy 0.9851

By class:
              precision    recall  f1-score   support

          공통     0.9688    1.0000    0.9841        31
       가설구조물     1.0000    1.0000    1.0000        10
          교량     1.0000    1.0000    1.0000         8
        양중작업     1.0000    0.8750    0.9333         8
        굴착공사     1.0000    1.0000    1.0000         7
          건축     1.0000    1.0000    1.0000         3

    accuracy                         0.9851        67
   macro avg     0.9948    0.9792    0.9862        67
weighted avg     0.9855    0.9851    0.9847        67

2025-05-01 08:36:46,360 ----------------------------------------------------------------------------------------------------


  훈련 결과: {'test_score': 0.9850746268656716}
  현재 폴드 정확도: 1.0000
  평균 성능: 0.9905 (유효한 폴드: 5/5)

=== 랜덤 서치 결과 요약 ===
1위: lr=0.0344, batch_size=32, epochs=19 - 점수: 0.9905 (유효 폴드: 5/5)
2위: lr=0.0503, batch_size=16, epochs=8 - 점수: 0.8916 (유효 폴드: 5/5)

최적 하이퍼파라미터: {'learning_rate': np.float64(0.03438585961287222), 'batch_size': 32, 'epochs': 19}, 점수: 0.9905


random_state=42의 기능:

- 재현성 보장: 같은 코드를 여러 번 실행해도 항상 동일한 데이터 분할 결과가 나오도록 함
무작위성 제어: 데이터를 섞는(shuffle) 과정에서 난수 생성기의 시드 값을 고정 -> shuffle = true의 기능은 random 알고리즘의 난수값 고정여부를 참으로 설정 , 그 고정 난수값 random state = 42

- 실험 비교 용이: 하이퍼파라미터만 바꿔서 실험해도 같은 데이터 분할을 사용하므로 공정한 비교 가능
값의 의미: 42라는 숫자 자체는 특별한 의미가 없으며, 관례적으로 많이 사용되는 값임

create_stratified_folds 함수의 기능:

- 계층적 데이터 분할: 각 클래스의 비율을 모든 폴드에 동일하게 유지하며 분할(우리의 훈련용 데이터를 그냥 나누기에는 비슷한 category들끼리 모여있어 어려움이 있음

 -> 훈련용 데이터 fold 5개를 각 label 별로 일정한
비율로 생성
-> 그리고 그 fold 안에서 또 훈련용/시험용 데이터 분할)

데이터 균형 유지:

- 예: 원본 데이터에 '교량공사' 30%, '터널공사' 20%, '도로공사' 50%의 비율이라면
각 폴드에도 이 비율이 유지됨


샘플링 편향 방지:
- 특정 카테고리가 특정 폴드에만 집중되는 것을 방지

교차 검증 준비:
-  각 폴드별 훈련/검증 데이터 인덱스를 생성하여 교차 검증에 사용할 수 있게 함

이 함수는 교차 검증의 품질을 높이기 위한 핵심 단계로, 특히 클래스 불균형이 있는 텍스트 분류 데이터셋에서 매우 중요합니다.

그러면 이렇게 fold하는 scikit-learn 패키지를 사용했을때 그 결과는 어떻게 나오는가?

fold_indices 안에 들어있는 데이터는 다음과 같은 형식입니다:

python[

      (train_indices_fold1, val_indices_fold1),  # 1번 폴드
      (train_indices_fold2, val_indices_fold2),  # 2번 폴드
      (train_indices_fold3, val_indices_fold3),  # 3번 폴드
      (train_indices_fold4, val_indices_fold4),  # 4번 폴드
      (train_indices_fold5, val_indices_fold5)   # 5번 폴드
]

여기서:

- 각 폴드는 튜플로 표현됩니다: (훈련 인덱스 배열, 검증 인덱스 배열)

- 각 인덱스 배열은 numpy.ndarray 타입:

    train_indices_fold1 = numpy.array([0, 1, 3, 4, 6, 7, ...])
    val_indices_fold1 = numpy.array([2, 5, 8, ...])

인덱스 비율:

- 훈련 인덱스: 전체 데이터의 약 80% (5-fold의 경우)
- 검증 인덱스: 전체 데이터의 약 20% (5-fold의 경우)


각 인덱스의 의미:

- 인덱스 값은 원본 데이터(texts와 labels 리스트)에서의 위치를 가리키는 정수
- 예: train_indices_fold1[0] = 3이면 원본 데이터의 3번째 항목



이 인덱스를 사용하여 원본 데이터에서 각 폴드별로 훈련/검증 데이터를 추출하고 모델을 학습/평가할 수 있습니다.


위와 같은 과정을 통해서 교차검증을 위한 fold를 생성하고 , 각 교차검증에 사용할 하이퍼파라미터를 랜덤서치 하는 과정을 수행, 각 fold별로 train-file , dev-file , corpus 생성 , 분류생성 , tainer 인스턴스 생성 , 각 train 의 결과 즉 accuracy 구하는 과정을 수행,

각 fold별로 , 각 하이퍼 파라미터 별로 수행했을때 가장 높은 acuuracy를 찾음으로써 하이퍼 파라미터 최적화 완료.


In [ ]:
# 새로운 분류기 생성
final_classifier = TextClassifier(document_embeddings,
                                 label_dictionary=construction_corpus.make_label_dictionary(),
                                 label_type='class')

# 트레이너 생성
trainer = ModelTrainer(final_classifier, construction_corpus)

# 최종 모델 학습 (train+dev 데이터 모두 사용)
trainer.train('models/final_construction_model',
             learning_rate=best_params['learning_rate'],
             mini_batch_size=best_params['batch_size'],
             max_epochs=best_params['epochs'])

In [ ]:
# 최종 테스트 수행
test_result = trainer.final_test('models/final_construction_model/best-model.pt')
print(f"최종 테스트 성능: {test_result}") # 내가 랜덤서치 + 교차검증으로 추적한 최적의 하이퍼파라미터를 최초의 test 데이터에 적용했을때 나오는 성능평가점수

#교차검증의 효과는 파라미터 최적화 보다는 실제 성능평가값을 더 정확하게 나타낼 수 있다는 점에 포커스를 맞추는건가?

- 맞습니다. 교차검증(Cross-validation)의 주요 목적은 실제 성능평가값을 더 정확하게 추정하는 데 있습니다.

##교차검증의 핵심 가치는:

- 데이터 활용 효율성: 제한된 데이터셋을 최대한 활용하여 모델의 일반화 성능을 평가합니다.
- 성능 평가의 신뢰성: 단일 테스트셋에 의존할 때 발생할 수 있는 편향을 줄이고, 여러 분할에서의 평균 성능을 측정함으로써 더 안정적인 평가 지표를 얻습니다.
- 과적합 감지: 학습 데이터와 검증 데이터 간의 성능 차이를 통해 모델의 과적합 여부를 확인할 수 있습니다.

## 단일 테스트셋에 의존하면 편향이 많이 발생하는가?

- 네, 단일 테스트셋에 의존하면 여러 종류의 편향이 발생할 수 있습니다:

- 샘플링 편향: 특정 테스트셋이 전체 데이터 분포를 충분히 대표하지 못할 수 있습니다. 예를 들어, 우연히 쉬운 샘플이나 어려운 샘플이 많이 포함될 수 있습니다.
- 데이터 분할 운(luck): 단일 분할에서는 우연히 "운 좋은" 또는 "운 나쁜" 분할이 발생할 수 있습니다. 즉, 특정 분할에서 모델이 비정상적으로 좋거나 나쁜 성능을 보일 수 있습니다.
- 과적합에 취약: 단일 테스트셋을 반복적으로 사용하면, 무의식적으로 이 특정 테스트셋에 최적화된 모델을 선택하게 되어 실제 환경에서의 일반화 성능이 떨어질 수 있습니다.
- 이상치(outlier) 영향: 작은 테스트셋에서는 소수의 이상치가 전체 성능 평가에 큰 영향을 미칠 수 있습니다.

교차검증은 이러한 문제들을 여러 번의 다른 데이터 분할을 통해 평균화함으로써 완화합니다. 예를 들어, k-fold 교차검증에서는 데이터를 k개의 부분으로 나누고 k번의 다른 학습-테스트 조합으로 모델을 평가합니다. 이렇게 하면 모든 데이터 포인트가 한 번은 테스트 세트에 포함되므로 더 안정적이고 일반화된 성능 추정치를 얻을 수 있습니다.

In [ ]:
# 1. 지침 유형 분류 모델 학습
# 데이터셋 로드 및 준비
type_corpus = ClassificationCorpus("/content/sample_data/type_train_dev_test",  # 데이터 디렉토리 경로
                                 train_file='type_train.txt',  # 학습 데이터 파일
                                 dev_file='type_dev.txt',  # 검증 데이터 파일
                                 test_file='type_test.txt')  # 테스트 데이터 파일

#-> 이 밑에 label_type 이라는 다른 속성을 따로 설정을 하지 않았으니 우리의 type_train.txt 의 라벨은 기본 설정인 class 라벨의 타입으로 변환된다.

# 한국어에 특화된 KLUE BERT 모델을 문서 임베딩으로 사용
document_embeddings = TransformerDocumentEmbeddings('klue/bert-base')

# 지침 유형 분류기 모델 초기화
# label_dictionary는 학습 데이터에서 자동으로 생성됨
type_classifier = TextClassifier(document_embeddings,
                                label_dictionary=type_corpus.make_label_dictionary(label_type= 'class'), label_type='class')
# 마지막의 label_type = 'class' 의 의미는 예측 결과로 나올 값의 타입이 class 라는 것

# 모델 트레이너 초기화
type_trainer = ModelTrainer(type_classifier, type_corpus)

# 모델 학습 시작
# models/type-classifier 경로에 모델 저장
type_trainer.train('models/type-classifier',
                  learning_rate=0.02,  # 학습률 설정
                  mini_batch_size=16,  # 배치 크기 설정
                  max_epochs=10,
                   )  # 최대 에폭 수 설정

# 지침 유형 중 위험 경고의 데이터가 너무 적어서 그런지 -> accuracy 가 지나치게 낮게 나옴.(train용 데이터가 더 많이 필요할 듯.)

# 샘플 데이터 출력해보기
print("훈련 데이터 샘플:")
for i, sentence in enumerate(type_corpus.train):
    print(f"문장 {i}: {sentence.text}")
    print(f"레이블: {sentence.get_label('class')}")
    if i >= 2: break  # 처음 3개만 출력

2025-05-01 04:13:12,303 Reading data from /content/sample_data/type_train_dev_test
2025-05-01 04:13:12,304 Train: /content/sample_data/type_train_dev_test/type_train.txt
2025-05-01 04:13:12,305 Dev: /content/sample_data/type_train_dev_test/type_dev.txt
2025-05-01 04:13:12,306 Test: /content/sample_data/type_train_dev_test/type_test.txt
2025-05-01 04:13:12,353 Initialized corpus /content/sample_data/type_train_dev_test (label type name is 'class')
2025-05-01 04:13:15,483 Computing label dictionary. Progress:


0it [00:00, ?it/s]
838it [00:00, 2787.82it/s]

2025-05-01 04:13:15,791 Dictionary created for label 'class' with 3 values: 예방조치 (seen 419 times), 점검사항 (seen 325 times), 위험경고 (seen 94 times)
2025-05-01 04:13:15,860 ----------------------------------------------------------------------------------------------------
2025-05-01 04:13:15,862 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): 


/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:107: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-05-01 04:13:16,847 epoch 1 - iter 5/53 - loss 1.74800799 - time (sec): 0.96 - samples/sec: 83.17 - lr: 0.020000 - momentum: 0.000000
2025-05-01 04:13:18,042 epoch 1 - iter 10/53 - loss 1.90164078 - time (sec): 2.16 - samples/sec: 74.19 - lr: 0.020000 - momentum: 0.000000
2025-05-01 04:13:19,141 epoch 1 - iter 15/53 - loss 1.69551675 - time (sec): 3.26 - samples/sec: 73.71 - lr: 0.020000 - momentum: 0.000000
2025-05-01 04:13:19,984 epoch 1 - iter 20/53 - loss 1.61677293 - time (sec): 4.10 - samples/sec: 78.06 - lr: 0.020000 - momentum: 0.000000
2025-05-01 04:13:20,896 epoch 1 - iter 25/53 - loss 1.55796776 - time (sec): 5.01 - samples/sec: 79.83 - lr: 0.020000 - momentum: 0.000000
2025-05-01 04:13:21,878 epoch 1 - iter 30/53 - loss 1.53239835 - time (sec): 5.99 - samples/sec: 80.09 - lr: 0.020000 - momentum: 0.000000
2025-05-01 04:13:22,982 epoch 1 - iter 35/53 - loss 1.45984234 - time (sec): 7.10 - samples/sec: 78.91 - lr: 0.020000 - momentum: 0.000000
2025-05-01 04:13:24,057 epoc

...

100%|██████████| 2/2 [00:00<00:00,  3.73it/s]

2025-05-01 04:15:15,305 DEV : loss 1.516379475593567 - f1-score (micro avg)  0.7143
2025-05-01 04:15:15,346  - 0 epochs without improvement
2025-05-01 04:15:15,349 saving best model


2025-05-01 04:15:18,575 ----------------------------------------------------------------------------------------------------
2025-05-01 04:15:18,578 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  3.42it/s]


2025-05-01 04:15:21,782 
Results:
- F-score (micro) 0.6857
- F-score (macro) 0.5945
- Accuracy 0.6857

By class:
              precision    recall  f1-score   support

        예방조치     0.7925    0.7241    0.7568        58
        점검사항     0.6341    0.7647    0.6933        34
        위험경고     0.3636    0.3077    0.3333        13

    accuracy                         0.6857       105
   macro avg     0.5967    0.5988    0.5945       105
weighted avg     0.6881    0.6857    0.6838       105

2025-05-01 04:15:21,783 ----------------------------------------------------------------------------------------------------
훈련 데이터 샘플:
문장 0: 철거작업에 사용되는 수동 및 전동기계․기구에 대하여는 반입 전에 이상 유무 점검을 실시한다.
레이블: Sentence[13]: "철거작업에 사용되는 수동 및 전동기계․기구에 대하여는 반입 전에 이상 유무 점검을 실시한다." → 점검사항 (1.0)
문장 1: 굴착작업 진행 시 일반 슬래브는 토압, 수압 등의 횡력에 버팀 역할을 하나, 램프 슬래브는 상하층 경사로로 시공되어 구조물이 완성되기 전까지는 횡력에 대한 버팀대의 역할을 하지 못하므로 횡력에 저항토록 버팀대(Strut)등을 별도로 설치 하여 붕괴 및 변형을 방지하도록 하여야 한다.
레이블: Sentence[43]: "굴착작업 진행 시 일반 슬래브는 토압, 수압 등의 횡력에 버팀 역할을 하나

In [ ]:
!pip install git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-wu64qljk
  Running command git clone --filter=blob:none --quiet https://github.com/flairNLP/flair.git /tmp/pip-req-build-wu64qljk
  Resolved https://github.com/flairNLP/flair.git to commit ee8596c2bbe737ec9ddeb1c6cb62fa0b161f4d84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import flair
print(flair.__version__)

0.15.1


In [ ]:
# 2. 공사 종류 분류 모델 학습
# 데이터셋 로드 및 준비
construction_corpus = ClassificationCorpus("/content/sample_data/construction_train_dev_test",  # 데이터 디렉토리 경로
                                         train_file='construction_train.txt',  # 학습 데이터 파일
                                         dev_file='construction_dev.txt',  # 검증 데이터 파일
                                         test_file='construction_test.txt', # 테스트 데이터 파일
                                         )
# test.txt에 있는 레이블 확인하기
label_counts = {}

# 테스트 데이터의 각 문장을 순회하며 레이블 카운트
for sentence in construction_corpus.train:
    label = sentence.get_label('class').value
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1

# 결과 출력
print("=== 테스트 데이터(test.txt) 레이블 분포 ===")
for label, count in sorted(label_counts.items()):
    percentage = (count / sum(label_counts.values())) * 100
    print(f"레이블: {label} - {count}개 ({percentage:.2f}%)")

print(f"총 샘플 수: {sum(label_counts.values())}")
print(f"고유 레이블 수: {len(label_counts)}")
print(f"레이블 목록: {list(label_counts.keys())}")

# test.txt에 있는 레이블 확인하기
label_counts = {}

# 테스트 데이터의 각 문장을 순회하며 레이블 카운트
for sentence in construction_corpus.test:
    label = sentence.get_label('class').value
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1

# 결과 출력
print("=== 테스트 데이터(test.txt) 레이블 분포 ===")
for label, count in sorted(label_counts.items()):
    percentage = (count / sum(label_counts.values())) * 100
    print(f"레이블: {label} - {count}개 ({percentage:.2f}%)")

print(f"총 샘플 수: {sum(label_counts.values())}")
print(f"고유 레이블 수: {len(label_counts)}")
print(f"레이블 목록: {list(label_counts.keys())}")

# 앞서 생성한 document_embeddings를 재사용 (모델 공유)
construction_classifier = TextClassifier(document_embeddings,
                                       label_dictionary=construction_corpus.make_label_dictionary(label_type= 'class'), label_type='class')  # 공사 유형 레이블 사전 생성

# 모델 트레이너 초기화
construction_trainer = ModelTrainer(construction_classifier, construction_corpus)

construction_trainer.train('models/construction-classifier',
                  learning_rate=0.0382,  # 학습률 설정
                  mini_batch_size=16,  # 배치 크기 설정
                  max_epochs=18
                  )  # 최대 에폭 수 설정

print("레이블 분포:", label_counts)


2025-05-01 06:11:02,201 Reading data from /content/sample_data/construction_train_dev_test
2025-05-01 06:11:02,202 Train: /content/sample_data/construction_train_dev_test/construction_train.txt
2025-05-01 06:11:02,203 Dev: /content/sample_data/construction_train_dev_test/construction_dev.txt
2025-05-01 06:11:02,205 Test: /content/sample_data/construction_train_dev_test/construction_test.txt
2025-05-01 06:11:02,253 Initialized corpus /content/sample_data/construction_train_dev_test (label type name is 'class')
=== 테스트 데이터(test.txt) 레이블 분포 ===
레이블:  - 5개 (0.60%)
레이블: 가설구조물 - 198개 (23.63%)
레이블: 건축 - 77개 (9.19%)
레이블: 공통 - 244개 (29.12%)
레이블: 교량 - 132개 (15.75%)
레이블: 굴착공사 - 109개 (13.01%)
레이블: 양중작업 - 73개 (8.71%)
총 샘플 수: 838
고유 레이블 수: 7
레이블 목록: ['공통', '굴착공사', '가설구조물', '건축', '교량', '양중작업', '']
=== 테스트 데이터(test.txt) 레이블 분포 ===
레이블: 가설구조물 - 24개 (22.86%)
레이블: 건축 - 12개 (11.43%)
레이블: 공통 - 25개 (23.81%)
레이블: 교량 - 27개 (25.71%)
레이블: 굴착공사 - 11개 (10.48%)
레이블: 양중작업 - 6개 (5.71%)
총 샘플 수: 105
고유 레이블 수: 6
레이블 목록

0it [00:00, ?it/s]
838it [00:00, 3742.91it/s]

2025-05-01 06:11:02,627 Dictionary created for label 'class' with 7 values: 공통 (seen 244 times), 가설구조물 (seen 198 times), 교량 (seen 132 times), 굴착공사 (seen 109 times), 건축 (seen 77 times), 양중작업 (seen 73 times),  (seen 5 times)
2025-05-01 06:11:02,634 ----------------------------------------------------------------------------------------------------
2025-05-01 06:11:02,635 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (quer


/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-05-01 06:11:03,581 epoch 1 - iter 5/53 - loss 0.70823773 - time (sec): 0.92 - samples/sec: 86.70 - lr: 0.038200 - momentum: 0.000000
2025-05-01 06:11:04,484 epoch 1 - iter 10/53 - loss 0.35816162 - time (sec): 1.83 - samples/sec: 87.67 - lr: 0.038200 - momentum: 0.000000
2025-05-01 06:11:05,347 epoch 1 - iter 15/53 - loss 0.25857840 - time (sec): 2.69 - samples/sec: 89.29 - lr: 0.038200 - momentum: 0.000000
2025-05-01 06:11:06,271 epoch 1 - iter 20/53 - loss 0.19517475 - time (sec): 3.61 - samples/sec: 88.59 - lr: 0.038200 - momentum: 0.000000
2025-05-01 06:11:07,252 epoch 1 - iter 25/53 - loss 0.15988458 - time (sec): 4.59 - samples/sec: 87.09 - lr: 0.038200 - momentum: 0.000000
2025-05-01 06:11:08,270 epoch 1 - iter 30/53 - loss 0.13336549 - time (sec): 5.61 - samples/sec: 85.54 - lr: 0.038200 - momentum: 0.000000
2025-05-01 06:11:09,248 epoch 1 - iter 35/53 - loss 0.11437082 - time (sec): 6.59 - samples/sec: 84.98 - lr: 0.038200 - momentum: 0.000000
2025-05-01 06:11:10,296 epoc

...

100%|██████████| 2/2 [00:01<00:00,  1.85it/s]

2025-05-01 06:14:27,644 DEV : loss 3.6342153549194336 - f1-score (micro avg)  0.6381
2025-05-01 06:14:27,665  - 1 epochs without improvement


2025-05-01 06:14:30,489 ----------------------------------------------------------------------------------------------------
2025-05-01 06:14:30,494 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  3.40it/s]

2025-05-01 06:14:33,301 
Results:
- F-score (micro) 0.6857
- F-score (macro) 0.5175
- Accuracy 0.6857

By class:
              precision    recall  f1-score   support

          교량     0.6875    0.8148    0.7458        27
          공통     0.6897    0.8000    0.7407        25
       가설구조물     0.8571    0.7500    0.8000        24
        굴착공사     0.5385    0.6364    0.5833        11
          건축     0.6000    0.2500    0.3529        12
        양중작업     0.5000    0.3333    0.4000         6
                 0.0000    0.0000    0.0000         0

    accuracy                         0.6857       105
   macro avg     0.5533    0.5121    0.5175       105
weighted avg     0.6905    0.6857    0.6753       105

2025-05-01 06:14:33,301 ----------------------------------------------------------------------------------------------------
레이블 분포: {'공통': 25, '가설구조물': 24, '교량': 27, '굴착공사': 11, '건축': 12, '양중작업': 6}


학습 단계:

- 코퍼스에 있는 텍스트 데이터들을 document_embeddings를 통해 벡터화합니다.
- 각 벡터와 해당 텍스트의 라벨('label'로 시작하는) 간의 관계를 학습합니다.
- 이 과정에서 각 클래스(라벨)에 해당하는 벡터 공간의 특성을 파악합니다.


예측 단계:

- 새로운 텍스트가 입력되면, 같은 document_embeddings를 사용해 벡터로 변환합니다.
- 이 벡터가 학습 데이터의 어떤 클래스 벡터들과 가장 유사한지 계산합니다.
- 가장 유사도가 높은 클래스를 예측 결과로 출력합니다.



이는 마치 벡터 공간에서 각 클래스별로 데이터 포인트들이 모여있는 영역(클러스터)이 형성되고, 새로운 데이터가 어떤 클러스터에 가장 가까운지를 판단하는 것과 유사합니다. 라벨 사전(label_dictionary)은 이 클래스들의 정보를 관리하는 역할을 합니다.

--------------------------------------

학습률(learning_rate=0.02):

- 모델이 학습 과정에서 가중치를 업데이트하는 스텝 크기를 결정합니다.
  값이 크면 학습이 빠르게 진행되지만 최적점을 지나칠 수 있고, 작으면 학습이 느리지만 더 정확한 최적점에 도달할 가능성이 높아집니다.



배치 크기(mini_batch_size=16):

- 한 번에 처리되는 학습 데이터 샘플의 수입니다.
메모리 사용량과 학습 안정성 사이의 균형을 맞추는 중요한 요소입니다.



최대 에폭 수(max_epochs=10):

- 전체 학습 데이터셋을 몇 번 반복해서 학습할지 결정합니다.
너무 적으면 학습이 부족할 수 있고, 너무 많으면 과적합이 발생할 수 있습니다.



이 매개변수들이 달라지면 모델 성능에 다음과 같은 영향을 미칩니다:

- 학습률 변경 시: 너무 높으면 학습이 불안정해지고, 너무 낮으면 학습이 매우 느리거나 지역 최적점에 갇힐 수 있습니다.
- 배치 크기 변경 시: 큰 배치는 더 안정적인 그래디언트를 제공하지만 메모리를 많이 사용하고 일반화 성능이 떨어질 수 있습니다. 작은 배치는 노이즈가 있지만 종종 더 나은 일반화를 제공합니다.
- 에폭 수 변경 시: 에폭이 너무 적으면 과소적합, 너무 많으면 과적합이 발생할 수 있습니다.

이러한 매개변수들은 모델의 학습 과정과 최종 성능에 직접적인 영향을 미치므로, 일반적으로 하이퍼파라미터 튜닝 과정을 통해 최적값을 찾는 것이 중요합니다.

learning-rate-visualization.svg

correct-epochs-error-graph.svg

가중치의 의미:

- 머신러닝 모델에서 가중치는 입력 데이터의 각 특성(feature)이 최종 출력에 얼마나 영향을 미치는지 결정하는 값입니다.
예를 들어 신경망에서는 뉴런 간의 연결 강도를 나타냅니다.


가중치 업데이트 과정:

- 초기에는 가중치가 랜덤한 값으로 설정됩니다.
모델이 훈련 데이터로 예측을 수행하고, 실제 값과 예측 값의 차이(오차)를 계산합니다.
이 오차를 최소화하는 방향으로 가중치를 조정합니다.
이 과정을 반복하며 모델이 점점 더 정확한 예측을 할 수 있도록 합니다.


업데이트 수식:

- 가장 기본적인 형태: 새 가중치 = 기존 가중치 - (학습률 × 오차에 대한 가중치의 영향도)
여기서 '오차에 대한 가중치의 영향도'는 경사(gradient)로 계산됩니다.
학습률(learning rate)은 한 번에 가중치를 얼마나 크게 조정할지 결정합니다.


에폭과의 관계:

- 한 에폭은 전체 훈련 데이터를 한 번 순회하는 것을 의미합니다.
각 에폭에서 모델은 훈련 데이터를 보고 가중치를 여러 번 업데이트합니다.
에폭을 반복할수록 모델의 가중치는 최적값에 가까워지게 됩니다.



이 가중치 업데이트 과정이 바로 머신러닝에서 말하는 '학습'의 본질. 모델이 데이터로부터 패턴을 학습한다는 것은 결국 이 가중치들을 데이터에 맞게 조정해나가는 과정입다.

In [ ]:
# 우리가 직접 입력한

from sentence_transformers import SentenceTransformer

# 두 모델 로드
type_classifier = TextClassifier.load('models/type-classifier')
construction_classifier = TextClassifier.load('models/construction-classifier')

def classify_guideline(text):
    # 지침 유형 분류
    type_sentence = Sentence(text)
    type_classifier.predict(type_sentence)
    guideline_type = type_sentence.labels[0].value
    type_confidence = type_sentence.labels[0].score

    # 공사 종류 분류
    construction_sentence = Sentence(text)
    construction_classifier.predict(construction_sentence)
    construction_type = construction_sentence.labels[0].value
    construction_confidence = construction_sentence.labels[0].score

    return {
        'text': text,
        'guideline_type': guideline_type,
        'construction_type': construction_type,
        'type_confidence': type_confidence,
        'construction_confidence': construction_confidence
    }

# PDF에서 추출한 모든 지침에 적용
classified_guidelines = []
for guideline in fcm_guidelines + ilm_guidelines:
    classification = classify_guideline(guideline)

    # 두 분류 모두 충분한 신뢰도를 가질 경우만 사용
    if classification['type_confidence'] > 0.7 and classification['construction_confidence'] > 0.7:
        # 임베딩 생성
        embedding = model.encode(guideline)
        classification['embedding'] = embedding
        classified_guidelines.append(classification)

IsADirectoryError: [Errno 21] Is a directory: 'models/type-classifier'